In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('all_stocks_5yr.csv')
# df = df.iloc[:30000]

In [3]:
df.head()

,date,open,high,low,close,volume,Name
0,2013-02-08,15.07,15.12,14.63,14.75,8407500,AAL
1,2013-02-11,14.89,15.01,14.26,14.46,8882000,AAL
2,2013-02-12,14.45,14.51,14.10,14.27,8126000,AAL
3,2013-02-13,14.30,14.94,14.25,14.66,10259500,AAL
4,2013-02-14,14.94,14.96,13.16,13.99,31879900,AAL


In [4]:
df.describe()

,open,high,low,close,volume
count,619029.000000,619032.000000,619032.000000,619040.000000,6.190400e+05
mean,83.023334,83.778311,82.256096,83.043763,4.321823e+06
std,97.378769,98.207519,96.507421,97.389748,8.693610e+06
min,1.620000,1.690000,1.500000,1.590000,0.000000e+00
25%,40.220000,40.620000,39.830000,40.245000,1.070320e+06
50%,62.590000,63.150000,62.020000,62.620000,2.082094e+06
75%,94.370000,95.180000,93.540000,94.410000,4.284509e+06
max,2044.000000,2067.990000,2035.110000,2049.000000,6.182376e+08


In [ ]:
df['target'] = 0
for idx, row in df.iterrows():
    today_close = df.iloc[idx]['close']
    try:
        tomorrow_high = df.iloc[idx+1]['high'] 
    except IndexError:
        break
    if (0.99*tomorrow_high) <= today_close:
        df.iat[idx, 7] = 0
    else:
        df.iat[idx, 7] = 1

In [ ]:
df.head()

,date,open,high,low,close,volume,Name,target
0,2013-02-08,15.07,15.12,14.63,14.75,8407500,AAL,1
1,2013-02-11,14.89,15.01,14.26,14.46,8882000,AAL,0
2,2013-02-12,14.45,14.51,14.10,14.27,8126000,AAL,1
3,2013-02-13,14.30,14.94,14.25,14.66,10259500,AAL,1
4,2013-02-14,14.94,14.96,13.16,13.99,31879900,AAL,1


In [ ]:
split_date ='2017-05-01'
df_training = df.loc[df['date'] <= split_date]
df_test = df.loc[df['date'] > split_date]

In [ ]:
len(df_training)

520897

In [ ]:
len(df_test)

98143

In [ ]:
df_training.head()

,date,open,high,low,close,volume,Name,target
0,2013-02-08,15.07,15.12,14.63,14.75,8407500,AAL,1
1,2013-02-11,14.89,15.01,14.26,14.46,8882000,AAL,0
2,2013-02-12,14.45,14.51,14.10,14.27,8126000,AAL,1
3,2013-02-13,14.30,14.94,14.25,14.66,10259500,AAL,1
4,2013-02-14,14.94,14.96,13.16,13.99,31879900,AAL,1


In [ ]:
y_train = pd.DataFrame(df_training['target'].reset_index().fillna(0))
x_train = df_training.drop(['target', 'date', 'Name'], 1).reset_index().fillna(0)
y_test = df_test['target'].reset_index().fillna(0)
x_test = pd.DataFrame(df_test.drop(['target', 'date', 'Name'], 1).reset_index().fillna(0))

In [ ]:
y_train = y_train.values[:,1]
x_train = x_train.values[:,1:]
y_test = y_test.values[:,1]
x_test = x_test.values[:,1:]

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import classification_report

In [ ]:
classifiers = [
    GaussianNB(),
    #  RidgeClassifier(tol=1e-2, solver="lsqr"),
    QuadraticDiscriminantAnalysis(),
    LinearDiscriminantAnalysis(),
    DecisionTreeClassifier(max_depth=5),
    KNeighborsClassifier(3, n_jobs=-1),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1, n_jobs=-1),
    AdaBoostClassifier(),
    GradientBoostingClassifier(),
    # SVC(kernel="linear", C=0.025, probability=True),
    # SVC(gamma=2, C=1, probability=True),
    # SVC(),
    MLPClassifier(alpha=1),
    # GaussianProcessClassifier(1.0 * RBF(1.0), n_jobs=-1),
]

In [ ]:
for clf in classifiers:
    print('_' * 80)
    print(clf.__class__.__name__)
    clf.fit(x_train, y_train)
    print('Train/test accuracy: ', clf.score(x_train, y_train), clf.score(x_test, y_test))
    print('Classification report of Test data')
    print(classification_report(y_test, clf.predict(x_test)))

________________________________________________________________________________
GaussianNB
Train/test accuracy:  0.609108902527755 0.6490427233730373
Classification report of Test data
              precision    recall  f1-score   support

           0       0.66      0.97      0.78     63974
           1       0.46      0.04      0.08     34169

   micro avg       0.65      0.65      0.65     98143
   macro avg       0.56      0.51      0.43     98143
weighted avg       0.59      0.65      0.54     98143

________________________________________________________________________________
QuadraticDiscriminantAnalysis
Train/test accuracy:  0.6153634979660086 0.6520281629866623
Classification report of Test data
              precision    recall  f1-score   support

           0       0.67      0.94      0.78     63974
           1       0.50      0.12      0.20     34169

   micro avg       0.65      0.65      0.65     98143
   macro avg       0.58      0.53      0.49     98143
weighted 

/usr/local/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
